In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Dataframe").getOrCreate()

In [3]:
training=spark.read.csv("test1.csv", header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age', 'Experience'], outputCol='Independent Feature')

In [8]:
output = assembler.transform(training)
output.show()

+---------+---+----------+------+-------------------+
|     Name|Age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [9]:
finalized_data = output.select(['Independent Feature', 'Salary'])
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [11]:
from pyspark.ml.regression import LinearRegression
##Train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Feature', labelCol='Salary')
regressor=regressor.fit(train_data)

In [12]:
##Coefficients
regressor.coefficients

DenseVector([-714.2857, 3485.7143])

In [13]:
##Intercepts
regressor.intercept

26857.142857139796

In [15]:
##Prediction
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()

+-------------------+------+-----------------+
|Independent Feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [30.0,8.0]| 25000|33314.28571428435|
|        [31.0,10.0]| 30000|39571.42857142652|
+-------------------+------+-----------------+

